# Projet CoCoMa

## 1. Introduction

### 1.1. Résumé de l'article

Dans leur article intitulé *Comparing direct algorithms for two-player fair
division of indivisible items – A computational study* publié en 2017, Kilgour et Vetschera présentent une comparaison de différents algorithmes pour l'allocation de ressources indivisibles entre deux agents.

Pour pouvoir comparer ces algorithmes entre eux, les auteurs commencent par définir des propriétés que peuvent posséder les allocations trouvées par les algorithmes, et qui se basent sur les préférences des agents. Ces propriétés permettent d'évaluer la qualité de l'allocation renvoyée. A noter que les allocations doivent toujours être équilibrées, c'est-à-dire que chaque agent se voit allouer le même nombre de ressources par un algorithme. 

Deux types de propriétés sont distinguées:
- les propriétés ordinales qui se basent sur les rangs des ressources allouées : la Pareto-optimalité, l'Envy-freeness et la propriété Min-Max,
- les propriétés de Borda, qui reposent sur des scores (ou utilités) de Borda associés aux rangs des ressources : Borda-somme maximale (BS), Borda Envy-freeness (BE) et Borda Max-min (BM).

Les auteurs reprennent 5 algorithmes de la littérature : 
- Sequential (Brams et Al. 2015), 
- Singles-doubles et Iterated singles-doubles (Brams et Al. 2016), 
- Bottom-Up (Brams et Taylor 1996),
- Trump (Pruhs et Woeginger 2012). 

Ils en introduisent 3 autres qui se basent sur ceux cités précedemment : 
- Restricted Sequential, 
- Modified SD, 
- IS.

Certains algorithmes garantissent des propriétés sur leurs allocations, par exemple l'algorithme bottom-up garantit les propriétés Max-min. 

Les algorithmes ont été testés pour 2 agents et différentes quantités de ressources paires allant de 4 à 12, pour toutes les combinaisons de préférences possibles. 



### 1.2. Objectifs

Dans le cadre de ce mini-projet de Cocoma 2019, nous allons chercher à étendre les résultats de l'article pour trois agents. 

Dans la suite de ce notebook, nous décrivons nos choix d'algorithmes et la manière dont nous avons adapté ces derniers ainsi que les propriétés pour le cas à trois agents.

## 2. Projet

### 2.1 Génération des problèmes

On génère les problèmes (=préférences de l'agent) en permutant la liste d'item. On interprète le tableau obtenu comme le rang auquel est classé chaque item. Par exemple avec [2,3,4,6,5,1] comme liste de préférences, le premier item est de rang 2 tandis que le dernier item est de rang 1 (=préféré).
Pour N items, un agent a N! préférences possibles. Pour 3 agents, en fixant les préférences de l'un d'eux à [1,2,3,4,5,6], on a donc (N!)² problèmes possibles.

In [27]:
from situation3 import *

sit=Situation3(n_items=6)

print("Nombre de permutations possibles à "+str(sit._n_items)+" pour un agent = "+str(len(sit._preferences)))


Nombre de permutation de préférences possible à 6 pour un agent = 720


### 2.2 Génération des allocations

On génère toutes les allocations possibles à N items en générant les combinaisons possibles de taille N/3 des items disponibles et en faisant toutes les combinaisons par triplet de ces dernières.
Pour 3 agents et N items, on génère (N/3 parmi N)*(N/3 parmi 2N/3) allocations. 
Dans le cas 6 items on obtient:

In [26]:
#sit._problem.generateAllAlloc()
print("Nombre d'allocations possibles = "+str(len(sit._problem._allocations)))

Nombre d'allocations possibles = 90


### 2.3 Propriétés à 3 agents

Nous avons choisi d'adapter les propriétés de Borda pour évaluer nos allocations à 3 agents. La propriété Borda Pareto optimal (BP) étant induite par la propriété maximal Borda sum, elle n'a pas été implémentée. 
On désigne par $B_m$ le score de Borda de l'agent m.

In [9]:
from bordaProperties import *

#Création d'un agent:
#Agent([préférences],[allocations])
#l'élément à la position n de [préférences] correspond au rang du n+1 item

agents = []
agents.append(Agent([1,2,3,4,5,6],[1,2])) 
agents.append(Agent([3,4,1,5,6,2],[3,4]))
agents.append(Agent([3,2,5,1,4,6],[5,6]))

i=0
for a in agents:
    print("Score agent "+str(i)+": "+str(bordaScore(a)))
    i+=1

Score agent 0: 11
Score agent 1: 8
Score agent 2: 4


#### 2.3.1 Maximal Borda sum (BS)

Pour la propriété du maximum de la somme Borda on garde la même formulation que l'article avec A l'ensemble des allocations possibles pour un problème donné et m les 3 agents considérés:

$$\sum_{m=M}B_m(X_m)=\max_{Y \in A}\sum_{m=M}B_m(Y_m) $$

Pour montrer qu'une allocation vérifie cette propriété il faut donc calculer le score maximum qu'on obtient pour un agent et pour un problème donné en itérant sur toutes les allocations possibles. Si un agent a une allocation qui lui renvoie ce score alors cette allocation vérifie BS.

#### 2.3.2 Borda-envy-free (BE)

Pour la propriété d'envy-free, on considère qu'elle est vérifiée si tout agent n'est pas envieux de l'allocation des autres en considérant les scores de Borda en fonction de ses préférences. On a donc pour chacun des 3 agents noté m:

$$B_m(X_m) ≥ B_m(X_\overline{m}) $$

Avec $\overline{m}$ les 2 autres agents.

In [10]:
bordaEnvyWho(agents)

Agent 2 is jealous of agent 0.
Agent 2 is jealous of agent 1.


#### 2.3.3 Borda max-min (BM)

La propriété max-min Borda maximise le score du pire agent. Comme BS, elle ne change pas pour 3 agents:

$$\min_{m \in M}B_m(X_m) = \max_{Y \in A}min_{m \in M}B_m(Y_m)$$

### 2.4 Algorithmes à 3 agents

#### 2.4.1 Algorithme séquentiel (OS)

L'algorithme séquentiel pour trois agents reprend celui proposé par Brams et Al. (2015) pour deux agents.
Dans l'algorithme initial, on considère toutes les paires d'objets (Ia, Ib) dans Ha(L) et Hb(L) (respectivement le set d'objets non alloués et de rang L ou mieux, pour l'agent 'a' et l'agent 'b').
Dans notre version, on considère à la place tous les triplets d'objets (Ia, Ib, Ic) dans Ha(L), Hb(L) et Hc(L).

In [13]:
from algorithm import *
from agent import Agent

agents = []
agents.append(Agent([1,2,3,4,5,6],[]))
agents.append(Agent([3,4,1,5,6,2],[]))
agents.append(Agent([3,2,5,1,4,6],[]))

print("Sequential algorithm for 3 agents")
res = OS_3(agents)

print("---")
print("Possible allocations: ")
print(res)
print("Total number of allocations = "+str(len(res)))

Sequential algorithm for 3 agents
---
Possible allocations: 
[((1, 4), (3, 6), (2, 5)), ((1, 5), (3, 6), (2, 4)), ((1, 3), (4, 6), (2, 5)), ((1, 5), (4, 6), (2, 3)), ((1, 2), (4, 6), (3, 5)), ((1, 5), (4, 6), (2, 3)), ((1, 2), (4, 6), (3, 5)), ((2, 5), (4, 6), (1, 3))]
Total number of allocations = 8


#### 2.4.2 Algorithme Bottom-Up (BU)

L'algorithme bottom-up pour trois agents reprend celui proposé par Brams et Taylor (1996) pour deux agents, où un agent donne toujours à son adversaire l'objet non alloué qu'il préfère le moins.
Dans notre cas, chaque agent présente non pas un, mais deux adversaires. Ainsi, pour un tour, les agents peuvent passer l'objet non alloué non préférable soit à leur voisin de gauche, soit à leur voisin de droite.
De plus, on a trois choix possibles pour désigner l'agent qui commence. L'algorithme à trois agents devrait donc généré un nombre d'allocations bien plus important que celui à deux agents.

In [12]:
from algorithm import *
from agent import Agent

agents = []
agents.append(Agent([1,2,3,4,5,6],[]))
agents.append(Agent([3,4,1,5,6,2],[]))
agents.append(Agent([3,2,5,1,4,6],[]))

print("Bottom Up algorithm for 3 agents")
res = bottomUp_3(agents)

print("---")
print("Possible allocations: ")
print(res)
print("Total number of allocations = "+str(len(res)))

Bottom Up algorithm for 3 agents
---
Possible allocations: 
[((1, 2), (3, 4), (5, 6)), ((1, 2), (3, 4), (5, 6)), ((2, 3), (4, 5), (1, 6)), ((2, 3), (4, 5), (1, 6)), ((1, 4), (3, 6), (2, 5)), ((1, 4), (3, 6), (2, 5)), ((3, 4), (5, 6), (1, 2)), ((3, 4), (5, 6), (1, 2)), ((1, 4), (2, 3), (5, 6)), ((1, 4), (2, 3), (5, 6)), ((3, 4), (2, 5), (1, 6)), ((3, 4), (2, 5), (1, 6)), ((1, 2), (3, 6), (4, 5)), ((1, 2), (3, 6), (4, 5)), ((2, 3), (5, 6), (1, 4)), ((2, 3), (5, 6), (1, 4)), ((3, 6), (1, 4), (2, 5)), ((3, 6), (1, 4), (2, 5)), ((1, 6), (4, 5), (2, 3)), ((1, 6), (4, 5), (2, 3)), ((3, 4), (1, 2), (5, 6)), ((3, 4), (1, 2), (5, 6)), ((1, 4), (2, 5), (3, 6)), ((1, 4), (2, 5), (3, 6)), ((4, 6), (3, 5), (1, 2)), ((4, 6), (3, 5), (1, 2)), ((3, 6), (1, 5), (2, 4)), ((3, 6), (1, 5), (2, 4)), ((4, 5), (2, 3), (1, 6)), ((4, 5), (2, 3), (1, 6)), ((3, 5), (1, 2), (4, 6)), ((3, 5), (1, 2), (4, 6)), ((3, 5), (1, 2), (4, 6)), ((3, 5), (1, 2), (4, 6)), ((1, 5), (2, 4), (3, 6)), ((1, 5), (2, 4), (3, 6)), ((2

## 3. Résultats

Pour rester dans des temps de résolution raisonnables, nous nous sommes réduits à N = 6 ressources.
Ci-dessous, une démonstration pour un nombre limité de problèmes. On peut fixer nb_iter = -1 pour lancer le programme sur tous les problèmes possibles (cela risque de prendre plusieurs minutes).

In [5]:
from situation3 import *

sit=Situation3(n_items=6)
sit.run(nb_iter = 40,verbose=False)

sit.printResults()

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
(Tab 1.1) Mean number of generated allocations
          6 items   
OS        14.88     
BU        9.98      

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
(Tab 1.2) Fraction of allocations with Borda properties
BS        BE        BM        Total     
6         6         4         3690      
0.16%     0.16%     0.11%     100.0%    

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
(Tab 1.3) Generated allocations with Borda properties
          None      BM        BE        BM+BE     BS        BS+BM     BS+BE     All       
None      2415      39        10        30        269       29        8         4         
OS        213       27        1         32        129       40        0         35        
BU        245       7         0         8         9         7         0         0         
Both      71        11        1         4         21        20        0         5         



Chacun des tableaux ci-dessus essaie de répondre à des questions énoncées par les auteurs dans l'article et qui permettent d'évaluer les algorithmes testés. 
Nous reprenons ces questions pour évaluer les algorithmes adaptés au cas trois agents :
- RQ1 : Combien d'allocations ont été générées en moyenne par les algorithmes ? 
- RQ2a : Quelle portion des allocations générées ont des propriétés désirables ?
- RQ2b : Quelle portion des allocations ayant des propriétés désirables ont été générées ?
- RQ2c : Dans quelle portion des problèmes les algorithmes arrivent à trouver au moins une allocation désirable (si elle existe) ?
- RQ3 : Quelle portion de bonnes allocations trouvées par un algorithme a été aussi trouvée par les autres algorithmes ?

La résolution complète sur tous les problèmes possibles a été sauvegardé dans un fichier csv.
En voici les résultats :

In [6]:
sit2 = Situation3(6)
sit2.loadResults("csv/complete_3.csv")

sit2.printResults()
sit2.printResultsAllProperties()


=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
(Tab 1.1) Mean number of generated allocations
          6 items   
OS        3.08      
BU        19.88     

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
(Tab 1.2) Fraction of allocations with Borda properties
BS        BE        BM        Total     
34320     3511808   124358    46656000  
0.07%     7.53%     0.27%     100.0%    

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
(Tab 1.3) Generated allocations with Borda properties
          None      BM        BE        BM+BE     BS        BS+BM     BS+BE     All       
None      32806343  37        2337165   48692     839       17        1466      14968     
OS        902300    31        217262    16661     773       30        0         4421      
BU        9075761   5         740851    21218     381       3         560       9614      
Both      357506    19        90302     7527      133       14        0         1101      

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

## 4. Discussion

Dans un premier temps, on voit tout de suite l'intérêt d'effectuer cette étude sur toutes les préférences possibles. En effet, les proportions d'allocations ayant des propriétés de Borda sont très différentes, suivant qu'on lance le programme sur une partie ou l'ensemble de problèmes (voir Tab 1.2 de la démonstration et des résultats finaux). 

Pour la première **question RQ1**, on observe sur le tableau (Tab 1.1) un nombre plus important d'allocations générées par rapport au cas à deux agents (voir Figure 1 de l'article). Si on se réfère aux données de l'article, pour N=6, on passe de 1.7 à 3.08 pour l'algorithme OS (+81%) et de 1.7 à 19.88 pour l'algorithme BU (+1070%).  
**L'algorithme OS** assigne des triplets d'objets, ce qui veut dire qu'à chaque moment où il y a contestation, on a (3!) embranchements. Ainsi, pour N=items, l'algorithme OS peut retourner au maximum (3!)^(N/3) allocations. Dans notre cas à N=6, nous avons donc 36 allocations tout au plus, ce qui est largement au dessus de 8 allocations maximales dans le cas 2 agents.  
**L'algorithme BU** crée deux branches à chaque tour (c'est à dire à chaque fois que les trois agents reçoivent un objet en plus). Pour N=6, nous avons 6/3 = 2 tours d'allocations. Il faut aussi prendre en compte les (3!) combinaisons d'ordre de passage des agents. Ainsi, l'algorithme renvoit tout au plus (3!)*2² = 24 allocations. 

Ensuite, évaluons les performances de ces algorithmes sur la base de propriétés désirables. Nous avons utilisé les propriétés  de Borda BS, BE et BM décritent 2.2.  
* Pour la **question RQ2a**, observons la table 1.3. On observe que 21.1% des allocations générées par OS et 8.45% par BU satisfont au moins une propriété. De plus, on observe que seulement 0.35% des allocations pour OS et 0.10% pour BU présentent toutes les propriétés de Borda (table 2.1). Nos versions des algorithmes pour le cas trois agents présenterait donc des performances bien moindre par rapport aux algorithmes originels pour deux agents (voir figure 3 et table 1 de l'article).  
On pourrait avancer que des allocations respectant les propriétés de Borda à trois agents sont plus difficiles à trouver. Si l'on observe la table 1.2, on voit que la proportion d'allocations ayant des propriétés de Borda  est bien moindre. Pour BE, on passe de ~25% à 7.53% (-70%). Pour BS, on passe de ~8.5% à 0.07% (-99.17%) et pour BM, on passe de 7.5% à 0.27% (-96.4%). Les propriétés BM et BS deviennent donc très fortes dans le cas à trois agents, ce qui expliquerait la baisse de performance apparente des algorithmes.  
Il pourrait être intéressant de modifier ces propriétés ou bien d'en trouver d'autres plus faibles et ainsi plus approprié pour l'études du cas à trois agents.  
* Pour la **question RQ2b**, on peut calculer à partir de la table 1.3 les pourcentages d'allocations ayant des propriétés qui ont pu être générées par les algorithmes. Nous obtenons le tableau ci-dessous :

|  .  | BS  | BE  | BM  | All  |
|:-----:|:-----:|:-----:|:-----:|:------:|
| OS  |  18.9%  |  9.6%  | 24.0%  |  18.3%  |
|  BU |  34.4%  | 24.8% |  31.8% | 35.6%  |

Les deux algorithmes parviennent à trouver une quantité non négligeable d'allocations avec de bonnes propriétés.
Cependant, on remarque certaines propriétés garanties dans le cas à 2 agents ne le sont plus dans le cas à 3 agents. 
Ainsi, l'algorithme OS ne garantit plus l'envy-freeness de même que l'algorithme BU ne garantit plus max-min.


* Pour la **question RQ2c**, observons la table 2.2. Malgré le caractère restrictif des propriétés évoqué plus tôt, on voit que les deux algorithmes trouvent souvent au moins une allocation satisfaisant les trois propriétés de Borda. Pour faire le lien avec la question RQ1, les deux algorithmes (surtout BU) génèrent plus d'allocations, ce qui augmente aussi les chances de tomber sur des allocations désirables.

Enfin pour la **question RQ3**, on peut étudier les allocations partagées / exclusives à partir des données de la table 1.3. En voici les résultats

|  .  | Exclusif  | Partagé | Manqué  | Tous manqués  |
|:-----:|:-----:|:-----:|:-----:|:------:|
| OS  |  14.7%  |  3.66%  | 31.9%  |  49.7%  |
|  BU |  31.9%  | 3.66% |  14.7% | 49.7%  |

On voit que les deux algorithmes présentent beaucoup d'allocations exclusives et en partagent peu. On peut avancer les deux algorithmes fonctionnent sur des principes bien distincts : OS est un algorithme top-down alors que BU est bottom-up. 

## 5. Conclusion

Pour conclure, nous avons dans un premier temps repris des travaux pour la répartition de ressources indivisibles entre deux agents et nous avons essayé de les adapter pour le cas à trois agents. 
Plus précisément, nous avons repris les algorithmes Sequential et Bottom-up et nous avons utilisé trois propriétés basées sur le score de Borda (Borda-somme maximale, Borda envy-freeness, Borda max-min) pour évaluer les résultats.  sur l'ensemble de problèmes possibles avec 6 ressources à partager. 
Le nombre d'items a été limiter par la puissance de nos ordinateurs. Avec une étude sur 9 et 12 items aumoins, nous aurion pu observer l'évolution des résultats renvoyés par les algorithmes en fonction du nombre d'item comme cela est dans l'article.

Les résultats nous ont montré que nos algorithmes adaptés au cas 3 agents présentent des performances moins concluantes. 
Les algorithmes trouvent en proportion moins d'allocations désirables que leur version pour 2 agents. 
Plusieurs raisons peuvent être évoquées et faire l'objet d'études plus approfondies :
* Les propriétés (surtout BS et BM) deviennent trop fortes lors qu'on travaille avec 3 agents. 
* Les algorithmes pourraient être plus contraints afin de réduire le nombre d'allocations sans aucune propriété.